## Test Reader Scan Mode

In [ ]:
from ng_ai import NebulaReader
from ng_ai import NebulaGraphConfig

config_dict = {
    "graphd_hosts": "graphd:9669",
    "metad_hosts": "metad0:9559",
    "user": "root",
    "password": "nebula",
    "space": "basketballplayer",
}
config = NebulaGraphConfig(**config_dict)
reader = NebulaReader(engine="spark", config=config)
reader.scan(edge="follow", props="degree")
df = reader.read()

In [ ]:
df.show(2)

## Test Algo Pagerank

In [ ]:
pr_result = df.algo.pagerank(reset_prob=0.15, max_iter=10)

In [ ]:
pr_result.show(5)

## Test Reader Query Mode

In [ ]:
from ng_ai import NebulaReader
from ng_ai.config import NebulaGraphConfig

# read data with spark engine, query mode
config_dict = {
    "graphd_hosts": "graphd:9669",
    "metad_hosts": "metad0:9559",
    "user": "root",
    "password": "nebula",
    "space": "basketballplayer",
}
config = NebulaGraphConfig(**config_dict)
reader = NebulaReader(engine="spark", config=config)
query = """
    MATCH ()-[e:follow]->()
    RETURN e LIMIT 100000
"""
reader.query(query=query, edge="follow", props="degree")
df = reader.read()  # this will take some time

In [ ]:
df.show(2)

## Test Writer insert mode in sink: `nebulagraph_vertex`

In [ ]:
from ng_ai import NebulaWriter
from ng_ai.config import NebulaGraphConfig


df_result = df.algo.label_propagation()

config_dict = {
    "graphd_hosts": "graphd:9669",
    "metad_hosts": "metad0:9559",
    "user": "root",
    "password": "nebula",
    "space": "basketballplayer",
}
config = NebulaGraphConfig(**config_dict)

writer = NebulaWriter(
    data=df_result, sink="nebulagraph_vertex", config=config, engine="spark"
)

# map column louvain into property cluster_id
properties = {"lpa": "cluster_id"}

writer.set_options(
    space="basketballplayer",
    tag="label_propagation",
    vid_field="_id",
    properties=properties,
    batch_size=256,
    write_mode="insert",
)
# write back to NebulaGraph
writer.write()

## Test Writer insert mode in sink: `nebulagraph_edge`

In [ ]:
df_result = df.algo.jaccard_similarity()

writer = NebulaWriter(
    data=df_result, sink="nebulagraph_edge", config=config, engine="spark"
)

# map column louvain into property cluster_id
properties = {"similarity": "similarity"}

writer.set_options(
    space="basketballplayer",
    edge_type="jaccard_similarity",
    src_id="srcId",
    dst_id="dstId",
    src_id_policy="",
    dst_id_policy="",
    properties=properties,
    batch_size=256,
    write_mode="insert",
)

# write back to NebulaGraph
writer.write()